<a href="https://colab.research.google.com/github/AIRobotics2030/AIRobotics2030/blob/main/WELLTRACK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile welltrack_app.py
import streamlit as st
import json
import os
import pandas as pd
from datetime import datetime, timedelta

DATA_FILE = "/content/welltrack_data.json"
DEVELOPER_SECRET = "WELLTRACK_DEV_2025"

def normalize(val):
    return str(val).strip().lower() if val is not None else ""

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
                return data.get("meals", []), data.get("symptoms", [])
        except:
            pass
    return [
        {"id": 1, "food": "Oatmeal with berries", "time": "2025-11-30T08:00", "portion": "1 bowl", "notes": "Felt energized"},
        {"id": 2, "food": "Grilled salmon with vegetables", "time": "2025-11-30T13:30", "portion": "6 oz", "notes": "No issues"},
        {"id": 3, "food": "Dairy ice cream", "time": "2025-11-29T20:00", "portion": "1 cup", "notes": "Stomach discomfort"}
    ], [
        {"id": 1, "symptom": "Stomach discomfort", "severity": 7, "time": "2025-11-30T14:00", "notes": "After lunch"},
        {"id": 2, "symptom": "Headache", "severity": 4, "time": "2025-11-29T21:00", "notes": "After ice cream"},
        {" id": 3, "symptom": "Fatigue", "severity": 3, "time": "2025-11-28T16:00", "notes": "General tiredness"}
    ]

def save_data(meals, symptoms):
    with open(DATA_FILE, "w") as f:
        json.dump({"meals": meals, "symptoms": symptoms}, f)

def is_meal_duplicate(new_meal, meals):
    key = (normalize(new_meal["food"]), normalize(new_meal["time"]), normalize(new_meal["portion"]))
    return any((normalize(m["food"]), normalize(m["time"]), normalize(m["portion"])) == key for m in meals)

def is_symptom_duplicate(new_symptom, symptoms):
    key = (normalize(new_symptom["symptom"]), normalize(new_symptom["time"]), new_symptom["severity"])
    return any((normalize(s["symptom"]), normalize(s["time"]), s["severity"]) == key for s in symptoms)

def generate_insights(meals, symptoms):
    insights = []
    dairy_symptoms = [s for s in symptoms if "ice cream" in normalize(s.get("notes", "")) or "stomach" in normalize(s.get("symptom", ""))]
    if dairy_symptoms:
        insights.append({
            "food": "Dairy products",
            "symptoms": ["Stomach discomfort", "Headache"],
            "confidence": "Medium",
            "action": "Consider reducing dairy intake and monitor symptoms"
        })
    healthy_meals = [m for m in meals if "oatmeal" in normalize(m["food"]) or "salmon" in normalize(m["food"])]
    if healthy_meals:
        insights.append({
            "food": "Whole foods (oats, fish)",
            "symptoms": ["Increased energy", "Better digestion"],
            "confidence": "High",
            "action": "Continue incorporating these foods regularly"
        })
    return insights

def format_datetime(dt_str):
    try:
        return datetime.fromisoformat(dt_str).strftime("%b %-d, %-I:%M %p")
    except:
        return dt_str

def in_date_range(item, start, end):
    try:
        dt = datetime.fromisoformat(item["time"])
        return start <= dt.date() <= end
    except:
        return True

# Initialize
if "initialized" not in st.session_state:
    meals_init, symptoms_init = load_data()
    st.session_state.meals = meals_init
    st.session_state.symptoms = symptoms_init
    st.session_state.initialized = True

# Auth
st.sidebar.title("🔐 Developer Access")
dev_code = st.sidebar.text_input("Enter code", type="password")
is_developer = (dev_code == DEVELOPER_SECRET)

tabs_list = ["Log", "Insights"]
if is_developer:
    tabs_list = ["Log", "Meals", "Symptoms", "Insights", "📤 Export"]

st.set_page_config(page_title="WellTrack", page_icon="❤️", layout="centered")
st.title("❤️ WellTrack")
st.caption("Track meals, symptoms, and health insights")

tabs = st.tabs(tabs_list)

# LOG TAB
with tabs[0]:
    st.header("📝 Daily Health Log")
    log_type = st.radio("Log", (" Meal 🍽️ ", " Symptom ⚠️ "), horizontal=True)

    if "Meal" in log_type:
        with st.form("meal_form"):
            food = st.text_input("Food")
            portion = st.text_input("Portion")
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes")
            if st.form_submit_button("✅ Add Meal"):
                new_meal = {"id": int(datetime.now().timestamp()), "food": food, "portion": portion, "time": time, "notes": notes}
                if is_meal_duplicate(new_meal, st.session_state.meals):
                    st.warning("⚠️ Duplicate meal skipped.")
                else:
                    st.session_state.meals.insert(0, new_meal)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Meal added!")
                st.rerun()
    else:
        with st.form("symptom_form"):
            symptom = st.text_input("Symptom")
            severity = st.slider("Severity (1–10)", 1, 10, 5)
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes")
            if st.form_submit_button("✅ Add Symptom"):
                new_symptom = {"id": int(datetime.now().timestamp()), "symptom": symptom, "severity": severity, "time": time, "notes": notes}
                if is_symptom_duplicate(new_symptom, st.session_state.symptoms):
                    st.warning("⚠️ Duplicate symptom skipped.")
                else:
                    st.session_state.symptoms.insert(0, new_symptom)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Symptom added!")
                st.rerun()

    st.markdown("### 📥 Upload Data")
    st.write("Upload your meal/symptom CSV (at least a month of data for better insights).")
    uploaded = st.file_uploader("CSV file", type=["csv"])
    if uploaded:
        df = pd.read_csv(uploaded)
        added_meals = 0
        added_symptoms = 0
        if {"food", "time", "portion", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                meal = {"id": int(datetime.now().timestamp() + added_meals), "food": str(row["food"]), "portion": str(row.get("portion", "")), "time": str(row.get("time", "")), "notes": str(row.get("notes", ""))}
                if not is_meal_duplicate(meal, st.session_state.meals):
                    st.session_state.meals.insert(0, meal)
                    added_meals += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_meals} new meals.")
            st.rerun()
        elif {"symptom", "severity", "time", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                symptom = {"id": int(datetime.now().timestamp() + added_symptoms), "symptom": str(row["symptom"]), "severity": int(row.get("severity", 5)), "time": str(row.get("time", "")), "notes": str(row.get("notes", ""))}
                if not is_symptom_duplicate(symptom, st.session_state.symptoms):
                    st.session_state.symptoms.insert(0, symptom)
                    added_symptoms += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_symptoms} new symptoms.")
            st.rerun()
        else:
            st.error("❌ Invalid format. Use templates.")

    st.markdown("### 📄 Templates")
    st.write("Download the sample and follow it in your data preparation.")
    col1, col2 = st.columns(2)
    col1.download_button("Meal Template", pd.DataFrame([{"food":"Apple","time":"2025-11-30T08:00","portion":"1","notes":"Felt good"}]).to_csv(index=False), "meal.csv")
    col2.download_button("Symptom Template", pd.DataFrame([{"symptom":"Headache","severity":5,"time":"2025-11-30T10:00","notes":"Stress-related"}]).to_csv(index=False), "symptom.csv")

# DEVELOPER-ONLY TABS
if is_developer:
    # MEALS TAB
    with tabs[1]:
        st.header("Meal History")
        colf1, colf2 = st.columns(2)
        max_date = datetime.today().date()
        start = colf1.date_input("Start", value=max_date - timedelta(days=30))
        end = colf2.date_input("End", value=max_date)
        filtered_meals = [m for m in st.session_state.meals if in_date_range(m, start, end)]
        for meal in filtered_meals:
            with st.expander(f"🍽️ {meal['food']} — {format_datetime(meal['time'])}"):
                st.write(f"**Portion:** {meal['portion']}")
                if meal["notes"]: st.write(f"_{meal['notes']}_")
                if st.button("🗑️ Delete", key=f"del_meal_{meal['id']}"):
                    st.session_state.meals = [m for m in st.session_state.meals if m["id"] != meal["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()

        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Meals (Keep First)", type="secondary"):
            seen = set()
            unique_meals = []
            dup_count = 0
            for m in st.session_state.meals:
                key = (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"]))
                if key not in seen:
                    unique_meals.append(m)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.meals = unique_meals
            save_data(st.session_state.meals, st.session_state.symptoms)
            if dup_count > 0:
                st.success(f"✅ Deleted {dup_count} duplicate meals!")
            else:
                st.info("No duplicates found.")
            st.rerun()

    # SYMPTOMS TAB
    with tabs[2]:
        st.header("Symptom Tracker")
        colf1, colf2 = st.columns(2)
        start = colf1.date_input("Start", value=max_date - timedelta(days=30), key="symptom_start")
        end = colf2.date_input("End", value=max_date, key="symptom_end")
        filtered_symptoms = [s for s in st.session_state.symptoms if in_date_range(s, start, end)]
        for s in filtered_symptoms:
            color = "🟢" if s["severity"] <= 3 else "🟡" if s["severity"] <= 7 else "🔴"
            with st.expander(f"{color} {s['symptom']} — {s['severity']}/10 — {format_datetime(s['time'])}"):
                if s["notes"]: st.write(f"_{s['notes']}_")
                if st.button("🗑️ Delete", key=f"del_symptom_{s['id']}"):
                    st.session_state.symptoms = [x for x in st.session_state.symptoms if x["id"] != s["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()

        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Symptoms (Keep First)", type="secondary"):
            seen = set()
            unique_symptoms = []
            dup_count = 0
            for s in st.session_state.symptoms:
                key = (normalize(s["symptom"]), normalize(s["time"]), s["severity"])
                if key not in seen:
                    unique_symptoms.append(s)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.symptoms = unique_symptoms
            save_data(st.session_state.meals, st.session_state.symptoms)
            if dup_count > 0:
                st.success(f"✅ Deleted {dup_count} duplicate symptoms!")
            else:
                st.info("No duplicates found.")
            st.rerun()

    # INSIGHTS TAB
    with tabs[3]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log more data to see insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

    # EXPORT TAB
    with tabs[4]:
        st.header("📤 Export")
        if st.session_state.meals:
            st.download_button("📥 Meals (CSV)", pd.DataFrame(st.session_state.meals).to_csv(index=False), "meals.csv")
        if st.session_state.symptoms:
            st.download_button("📥 Symptoms (CSV)", pd.DataFrame(st.session_state.symptoms).to_csv(index=False), "symptoms.csv")

else:
    with tabs[1]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log more data to see insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

Writing welltrack_app.py


In [4]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"\n✅ Your private WellTrack app is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &

ERROR:pyngrok.process.ngrok:t=2025-11-30T13:46:35+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-30T13:46:35+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-30T13:46:35+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour 

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [5]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok

# IMPORTANT: Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken.
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("36BkD6EggMaIWrI1jwhaN31aDAP_7vB143TdKTYt9zwPgyzTC")

public_url = ngrok.connect(8501)
print(f"\n✅ Your private WellTrack app is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &


✅ Your private WellTrack app is live at:
👉 NgrokTunnel: "https://trisyllabical-semitruthfully-junko.ngrok-free.dev" -> "http://localhost:8501"
🔓 Developer code: WELLTRACK_DEV_2025


In [6]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"\n✅ Your private WellTrack app is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &


✅ Your private WellTrack app is live at:
👉 NgrokTunnel: "https://trisyllabical-semitruthfully-junko.ngrok-free.dev" -> "http://localhost:8501"
🔓 Developer code: WELLTRACK_DEV_2025


In [12]:
%%writefile welltrack_app.py
import streamlit as st
import json
import os
import pandas as pd
from datetime import datetime, timedelta

DATA_FILE = "/content/welltrack_data.json"
DEVELOPER_SECRET = "WELLTRACK_DEV_2025"

def normalize(val):
    return str(val).strip().lower() if val is not None else ""

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
                meals = data.get("meals", [])
                symptoms = data.get("symptoms", [])

                # Ensure every meal has an 'id'
                for i, m in enumerate(meals):
                    if "id" not in m or m["id"] is None:
                        m["id"] = -1_000_000 - i

                # Ensure every symptom has an 'id'
                for i, s in enumerate(symptoms):
                    if "id" not in s or s["id"] is None:
                        s["id"] = -2_000_000 - i

                return meals, symptoms
        except Exception as e:
            st.error(f"Load error: {e}")

    # Return clean mock data with valid ids
    return [
        {"id": 1, "food": "Oatmeal with berries", "time": "2025-11-30T08:00", "portion": "1 bowl", "notes": "Felt energized"},
        {"id": 2, "food": "Grilled salmon with vegetables", "time": "2025-11-30T13:30", "portion": "6 oz", "notes": "No issues"},
        {"id": 3, "food": "Dairy ice cream", "time": "2025-11-29T20:00", "portion": "1 cup", "notes": "Stomach discomfort"}
    ], [
        {"id": 1, "symptom": "Stomach discomfort", "severity": 7, "time": "2025-11-30T14:00", "notes": "After lunch"},
        {"id": 2, "symptom": "Headache", "severity": 4, "time": "2025-11-29T21:00", "notes": "After ice cream"},
        {"id": 3, "symptom": "Fatigue", "severity": 3, "time": "2025-11-28T16:00", "notes": "General tiredness"}
    ]

def save_data(meals, symptoms):
    with open(DATA_FILE, "w") as f:
        json.dump({"meals": meals, "symptoms": symptoms}, f)

def is_meal_duplicate(new_meal, meals):
    key = (normalize(new_meal["food"]), normalize(new_meal["time"]), normalize(new_meal["portion"]))
    return any(
        (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"])) == key
        for m in meals
    )

def is_symptom_duplicate(new_symptom, symptoms):
    key = (normalize(new_symptom["symptom"]), normalize(new_symptom["time"]), new_symptom["severity"])
    return any(
        (normalize(s["symptom"]), normalize(s["time"]), s["severity"]) == key
        for s in symptoms
    )

def generate_insights(meals, symptoms):
    insights = []
    dairy_symptoms = [
        s for s in symptoms
        if "ice cream" in normalize(s.get("notes", "")) or "stomach" in normalize(s.get("symptom", ""))
    ]
    if dairy_symptoms:
        insights.append({
            "food": "Dairy products",
            "symptoms": ["Stomach discomfort", "Headache"],
            "confidence": "Medium",
            "action": "Consider reducing dairy intake and monitor symptoms"
        })
    healthy_meals = [
        m for m in meals
        if "oatmeal" in normalize(m["food"]) or "salmon" in normalize(m["food"])
    ]
    if healthy_meals:
        insights.append({
            "food": "Whole foods (oats, fish)",
            "symptoms": ["Increased energy", "Better digestion"],
            "confidence": "High",
            "action": "Continue incorporating these foods regularly"
        })
    return insights

def format_datetime(dt_str):
    try:
        return datetime.fromisoformat(dt_str).strftime("%b %-d, %-I:%M %p")
    except:
        return dt_str

def in_date_range(item, start, end):
    try:
        dt = datetime.fromisoformat(item["time"])
        return start <= dt.date() <= end
    except:
        return True

# Initialize
if "initialized" not in st.session_state:
    meals_init, symptoms_init = load_data()
    st.session_state.meals = meals_init
    st.session_state.symptoms = symptoms_init
    st.session_state.initialized = True

# Auth
st.sidebar.title("🔐 Developer Access")
dev_code = st.sidebar.text_input("Enter code", type="password")
is_developer = (dev_code == DEVELOPER_SECRET)

tabs_list = ["Log", "Insights"]
if is_developer:
    tabs_list = ["Log", "Meals", "Symptoms", "Insights", "📤 Export"]

st.set_page_config(page_title="WellTrack", page_icon="❤️", layout="centered")
st.title("❤️ WellTrack")
st.caption("Track meals, symptoms, and health insights")

tabs = st.tabs(tabs_list)

# LOG TAB
with tabs[0]:
    st.header("📝 Daily Health Log")
    log_type = st.radio("Log", (" Meal 🍽️ ", " Symptom ⚠️ "), horizontal=True)

    if "Meal" in log_type:
        with st.form("meal_form"):
            food = st.text_input("Food")
            portion = st.text_input("Portion")
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes")
            if st.form_submit_button("✅ Add Meal"):
                new_meal = {"id": int(datetime.now().timestamp()), "food": food, "portion": portion, "time": time, "notes": notes}
                if is_meal_duplicate(new_meal, st.session_state.meals):
                    st.warning("⚠️ Duplicate meal skipped.")
                else:
                    st.session_state.meals.insert(0, new_meal)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Meal added!")
                st.rerun()
    else:
        with st.form("symptom_form"):
            symptom = st.text_input("Symptom")
            severity = st.slider("Severity (1–10)", 1, 10, 5)
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes")
            if st.form_submit_button("✅ Add Symptom"):
                new_symptom = {"id": int(datetime.now().timestamp()), "symptom": symptom, "severity": severity, "time": time, "notes": notes}
                if is_symptom_duplicate(new_symptom, st.session_state.symptoms):
                    st.warning("⚠️ Duplicate symptom skipped.")
                else:
                    st.session_state.symptoms.insert(0, new_symptom)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Symptom added!")
                st.rerun()

    st.markdown("### 📥 Upload Data")
    st.write("Upload your meal/symptom CSV (at least a week of data for better insights).")
    uploaded = st.file_uploader("CSV file", type=["csv"])
    if uploaded:
        df = pd.read_csv(uploaded)
        added_meals = 0
        added_symptoms = 0
        if {"food", "time", "portion", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                meal = {
                    "id": int(datetime.now().timestamp() + added_meals),
                    "food": str(row["food"]),
                    "portion": str(row.get("portion", "")),
                    "time": str(row.get("time", "")),
                    "notes": str(row.get("notes", ""))
                }
                if not is_meal_duplicate(meal, st.session_state.meals):
                    st.session_state.meals.insert(0, meal)
                    added_meals += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_meals} new meals.")
            st.rerun()
        elif {"symptom", "severity", "time", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                symptom = {
                    "id": int(datetime.now().timestamp() + added_symptoms),
                    "symptom": str(row["symptom"]),
                    "severity": int(row.get("severity", 5)),
                    "time": str(row.get("time", "")),
                    "notes": str(row.get("notes", ""))
                }
                if not is_symptom_duplicate(symptom, st.session_state.symptoms):
                    st.session_state.symptoms.insert(0, symptom)
                    added_symptoms += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_symptoms} new symptoms.")
            st.rerun()
        else:
            st.error("❌ Invalid format. Use templates.")

    st.markdown("### 📄 Templates")
    st.write("Download the sample and follow it in your data preparation.")
    col1, col2 = st.columns(2)
    col1.download_button("Meal Template", pd.DataFrame([{"food":"Apple","time":"2025-11-30T08:00","portion":"1","notes":"Felt good"}]).to_csv(index=False), "meal.csv")
    col2.download_button("Symptom Template", pd.DataFrame([{"symptom":"Headache","severity":5,"time":"2025-11-30T10:00","notes":"Stress-related"}]).to_csv(index=False), "symptom.csv")

# DEVELOPER-ONLY TABS
if is_developer:
    # MEALS TAB
    with tabs[1]:
        st.header("Meal History")
        colf1, colf2 = st.columns(2)
        max_date = datetime.today().date()
        start = colf1.date_input("Start", value=max_date - timedelta(days=30))
        end = colf2.date_input("End", value=max_date)
        filtered_meals = [m for m in st.session_state.meals if in_date_range(m, start, end)]
        for meal in filtered_meals:
            with st.expander(f"🍽️ {meal['food']} — {format_datetime(meal['time'])}"):
                st.write(f"**Portion:** {meal['portion']}")
                if meal["notes"]: st.write(f"_{meal['notes']}_")
                if st.button("🗑️ Delete", key=f"del_meal_{meal['id']}"):
                    st.session_state.meals = [m for m in st.session_state.meals if m["id"] != meal["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()

        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Meals (Keep First)", type="secondary"):
            seen = set()
            unique_meals = []
            dup_count = 0
            for m in st.session_state.meals:
                key = (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"]))
                if key not in seen:
                    unique_meals.append(m)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.meals = unique_meals
            save_data(st.session_state.meals, st.session_state.symptoms)
            if dup_count > 0:
                st.success(f"✅ Deleted {dup_count} duplicate meals!")
            else:
                st.info("No duplicates found.")
            st.rerun()

    # SYMPTOMS TAB
    with tabs[2]:
        st.header("Symptom Tracker")
        colf1, colf2 = st.columns(2)
        start = colf1.date_input("Start", value=max_date - timedelta(days=30), key="symptom_start")
        end = colf2.date_input("End", value=max_date, key="symptom_end")
        filtered_symptoms = [s for s in st.session_state.symptoms if in_date_range(s, start, end)]
        for s in filtered_symptoms:
            color = "🟢" if s["severity"] <= 3 else "🟡" if s["severity"] <= 7 else "🔴"
            with st.expander(f"{color} {s['symptom']} — {s['severity']}/10 — {format_datetime(s['time'])}"):
                if s["notes"]: st.write(f"_{s['notes']}_")
                if st.button("🗑️ Delete", key=f"del_symptom_{s['id']}"):
                    st.session_state.symptoms = [x for x in st.session_state.symptoms if x["id"] != s["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()

        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Symptoms (Keep First)", type="secondary"):
            seen = set()
            unique_symptoms = []
            dup_count = 0
            for s in st.session_state.symptoms:
                key = (normalize(s["symptom"]), normalize(s["time"]), s["severity"])
                if key not in seen:
                    unique_symptoms.append(s)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.symptoms = unique_symptoms
            save_data(st.session_state.meals, st.session_state.symptoms)
            if dup_count > 0:
                st.success(f"✅ Deleted {dup_count} duplicate symptoms!")
            else:
                st.info("No duplicates found.")
            st.rerun()

    # INSIGHTS TAB
    with tabs[3]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log more data to see insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

    # EXPORT TAB
    with tabs[4]:
        st.header("📤 Export")
        if st.session_state.meals:
            st.download_button("📥 Meals (CSV)", pd.DataFrame(st.session_state.meals).to_csv(index=False), "meals.csv")
        if st.session_state.symptoms:
            st.download_button("📥 Symptoms (CSV)", pd.DataFrame(st.session_state.symptoms).to_csv(index=False), "symptoms.csv")

else:
    # USER VIEW: Only Log + Insights
    with tabs[1]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log more data to see insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

Overwriting welltrack_app.py


In [13]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"\n✅ WellTrack is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &


✅ WellTrack is live at:
👉 NgrokTunnel: "https://trisyllabical-semitruthfully-junko.ngrok-free.dev" -> "http://localhost:8501"
🔓 Developer code: WELLTRACK_DEV_2025


In [16]:
%%writefile welltrack_app.py
import streamlit as st
import json
import os
import pandas as pd
from datetime import datetime, timedelta

DATA_FILE = "/content/welltrack_data.json"
DEVELOPER_SECRET = "WELLTRACK_DEV_2025"

def normalize(val):
    return str(val).strip().lower() if val is not None else ""

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
                meals = data.get("meals", [])
                symptoms = data.get("symptoms", [])
                # Ensure every entry has a valid id
                for i, m in enumerate(meals):
                    if "id" not in m or m["id"] is None:
                        m["id"] = -1_000_000 - i
                for i, s in enumerate(symptoms):
                    if "id" not in s or s["id"] is None:
                        s["id"] = -2_000_000 - i
                return meals, symptoms
        except:
            pass
    # Return empty lists to start fresh (or add mock if desired)
    return [], []

def save_data(meals, symptoms):
    with open(DATA_FILE, "w") as f:
        json.dump({"meals": meals, "symptoms": symptoms}, f)

def is_meal_duplicate(new_meal, meals):
    key = (normalize(new_meal["food"]), normalize(new_meal["time"]), normalize(new_meal["portion"]))
    return any(
        (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"])) == key
        for m in meals
    )

def is_symptom_duplicate(new_symptom, symptoms):
    key = (normalize(new_symptom["symptom"]), normalize(new_symptom["time"]), new_symptom["severity"])
    return any(
        (normalize(s["symptom"]), normalize(s["time"]), s["severity"]) == key
        for s in symptoms
    )

def generate_insights(meals, symptoms):
    insights = []
    if not meals or not symptoms:
        return insights

    meal_text = " ".join(normalize(m["food"]) for m in meals)
    symptom_text = " ".join(normalize(s["symptom"]) + " " + normalize(s.get("notes", "")) for s in symptoms)

    # 🔸 Insight 1: Legumes (Beans, Ewa, Moimoo) → Gas/Bloating
    legume_foods = any(kw in meal_text for kw in ["bean", "ewa", "moimoo", "moi moi", "aganyin"])
    gas_symptoms = any(kw in symptom_text for kw in ["gas", "bloating", "cramp", "distension", "gassy", "flatulence"])
    if legume_foods and gas_symptoms:
        insights.append({
            "food": "Legume-based meals (Beans, Ewa Aganyin, Moimoo)",
            "symptoms": ["Gas", "Bloating", "Abdominal cramps"],
            "confidence": "High",
            "action": "Soak beans thoroughly before cooking and add digestive spices like ginger or garlic."
        })

    # 🔸 Insight 2: Fried/Palm Oil-Rich Foods → Heartburn
    fried_foods = any(kw in meal_text for kw in ["fried", "akara", "palm oil", "ayamase", "egusi", "pepper soup"])
    reflux_symptoms = any(kw in symptom_text for kw in ["heartburn", "reflux", "acid", "nausea", "burning"])
    if fried_foods and reflux_symptoms:
        insights.append({
            "food": "Fried or palm oil-rich dishes (Akara, Ayamase, Egusi)",
            "symptoms": ["Heartburn", "Acid reflux", "Nausea"],
            "confidence": "Medium",
            "action": "Reduce oil quantity or spice level; avoid eating late at night."
        })

    # 🔸 Insight 3: Swallow Foods (Amala, Eba, Fufu) → Heaviness
    swallow_foods = any(kw in meal_text for kw in ["amala", "eba", "fufu", "pounded yam", "lafun", "semo", "wheat meal"])
    fullness_symptoms = any(kw in symptom_text for kw in ["heavy", "full", "drowsy", "sluggish", "overly full", "sleepy"])
    if swallow_foods and fullness_symptoms:
        insights.append({
            "food": "Swallow meals (Amala, Eba, Pounded Yam)",
            "symptoms": ["Heaviness", "Drowsiness", "Digestive fullness"],
            "confidence": "Medium",
            "action": "Reduce portion size and pair with light, vegetable-based soups."
        })

    # 🔸 Insight 4: Spicy/Pepper-Heavy → Stomach Discomfort
    spicy_foods = any(kw in meal_text for kw in ["pepper", "spicy", "buka stew", "ofada stew"])
    stomach_symptoms = any(kw in symptom_text for kw in ["stomach", "discomfort", "ache", "pain", "upset"])
    if spicy_foods and stomach_symptoms:
        insights.append({
            "food": "Spicy or pepper-heavy stews",
            "symptoms": ["Stomach ache", "Discomfort", "Upset stomach"],
            "confidence": "Medium",
            "action": "Moderate pepper and spice levels, especially if you have a sensitive stomach."
        })

    return insights

def format_datetime(dt_str):
    try:
        return datetime.fromisoformat(dt_str).strftime("%b %-d, %-I:%M %p")
    except:
        return dt_str

def in_date_range(item, start, end):
    try:
        dt = datetime.fromisoformat(item["time"])
        return start <= dt.date() <= end
    except:
        return True

# Initialize session state
if "initialized" not in st.session_state:
    meals_init, symptoms_init = load_data()
    st.session_state.meals = meals_init
    st.session_state.symptoms = symptoms_init
    st.session_state.initialized = True

# Developer auth
st.sidebar.title("🔐 Developer Access")
dev_code = st.sidebar.text_input("Enter code", type="password")
is_developer = (dev_code == DEVELOPER_SECRET)

tabs_list = ["Log", "Insights"]
if is_developer:
    tabs_list = ["Log", "Meals", "Symptoms", "Insights", "📤 Export"]

st.set_page_config(page_title="WellTrack", page_icon="❤️", layout="centered")
st.title("❤️ WellTrack")
st.caption("Track Yoruba meals, symptoms, and health insights")

tabs = st.tabs(tabs_list)

# LOG TAB
with tabs[0]:
    st.header("📝 Daily Health Log")
    log_type = st.radio("Log", (" Meal 🍽️ ", " Symptom ⚠️ "), horizontal=True)

    if "Meal" in log_type:
        with st.form("meal_form"):
            food = st.text_input("Food", placeholder="e.g., Amala with Ewedu")
            portion = st.text_input("Portion", placeholder="e.g., 1 bowl, 2 wraps")
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes", placeholder="How did you feel? e.g., Felt bloated")
            if st.form_submit_button("✅ Add Meal"):
                new_meal = {"id": int(datetime.now().timestamp()), "food": food, "portion": portion, "time": time, "notes": notes}
                if is_meal_duplicate(new_meal, st.session_state.meals):
                    st.warning("⚠️ Duplicate meal skipped.")
                else:
                    st.session_state.meals.insert(0, new_meal)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Meal logged!")
                st.rerun()
    else:
        with st.form("symptom_form"):
            symptom = st.text_input("Symptom", placeholder="e.g., Stomach bloating")
            severity = st.slider("Severity (1–10)", 1, 10, 5)
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes", placeholder="e.g., After eating Ewa Aganyin")
            if st.form_submit_button("✅ Add Symptom"):
                new_symptom = {"id": int(datetime.now().timestamp()), "symptom": symptom, "severity": severity, "time": time, "notes": notes}
                if is_symptom_duplicate(new_symptom, st.session_state.symptoms):
                    st.warning("⚠️ Duplicate symptom skipped.")
                else:
                    st.session_state.symptoms.insert(0, new_symptom)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Symptom logged!")
                st.rerun()

    st.markdown("### 📥 Upload Data")
    st.write("Upload your Yoruba meal/symptom CSV (at least a month of data for better insights).")
    uploaded = st.file_uploader("CSV file", type=["csv"])
    if uploaded:
        df = pd.read_csv(uploaded)
        added_meals = 0
        added_symptoms = 0
        if {"food", "time", "portion", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                meal = {
                    "id": int(datetime.now().timestamp() + added_meals),
                    "food": str(row["food"]),
                    "portion": str(row.get("portion", "")),
                    "time": str(row.get("time", "")),
                    "notes": str(row.get("notes", ""))
                }
                if not is_meal_duplicate(meal, st.session_state.meals):
                    st.session_state.meals.insert(0, meal)
                    added_meals += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_meals} new meals.")
            st.rerun()
        elif {"symptom", "severity", "time", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                symptom = {
                    "id": int(datetime.now().timestamp() + added_symptoms),
                    "symptom": str(row["symptom"]),
                    "severity": int(row.get("severity", 5)),
                    "time": str(row.get("time", "")),
                    "notes": str(row.get("notes", ""))
                }
                if not is_symptom_duplicate(symptom, st.session_state.symptoms):
                    st.session_state.symptoms.insert(0, symptom)
                    added_symptoms += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_symptoms} new symptoms.")
            st.rerun()
        else:
            st.error("❌ Invalid format. Use templates.")

    st.markdown("### 📄 Templates")
    st.write("Download the sample and follow it in your data preparation.")
    col1, col2 = st.columns(2)
    col1.download_button("Meal Template", pd.DataFrame([{"food":"Amala with Ewedu","time":"2025-11-30T08:00","portion":"1 bowl","notes":"Felt heavy"}]).to_csv(index=False), "meal.csv")
    col2.download_button("Symptom Template", pd.DataFrame([{"symptom":"Stomach bloating","severity":6,"time":"2025-11-30T15:30","notes":"After Ewa Aganyin"}]).to_csv(index=False), "symptom.csv")

# DEVELOPER TABS
if is_developer:
    # MEALS TAB
    with tabs[1]:
        st.header("Meal History")
        colf1, colf2 = st.columns(2)
        max_date = datetime.today().date()
        start = colf1.date_input("Start", value=max_date - timedelta(days=30))
        end = colf2.date_input("End", value=max_date)
        filtered_meals = [m for m in st.session_state.meals if in_date_range(m, start, end)]
        for meal in filtered_meals:
            with st.expander(f"🍽️ {meal['food']} — {format_datetime(meal['time'])}"):
                st.write(f"**Portion:** {meal['portion']}")
                if meal["notes"]: st.write(f"_{meal['notes']}_")
                if st.button("🗑️ Delete", key=f"del_meal_{meal['id']}"):
                    st.session_state.meals = [m for m in st.session_state.meals if m["id"] != meal["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()
        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Meals (Keep First)", type="secondary"):
            seen = set()
            unique = []
            dup_count = 0
            for m in st.session_state.meals:
                key = (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"]))
                if key not in seen:
                    unique.append(m)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.meals = unique
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Deleted {dup_count} duplicates!") if dup_count else st.info("No duplicates.")
            st.rerun()

    # SYMPTOMS TAB
    with tabs[2]:
        st.header("Symptom Tracker")
        colf1, colf2 = st.columns(2)
        start = colf1.date_input("Start", value=max_date - timedelta(days=30), key="symptom_start")
        end = colf2.date_input("End", value=max_date, key="symptom_end")
        filtered = [s for s in st.session_state.symptoms if in_date_range(s, start, end)]
        for s in filtered:
            color = "🟢" if s["severity"] <= 3 else "🟡" if s["severity"] <= 7 else "🔴"
            with st.expander(f"{color} {s['symptom']} — {s['severity']}/10 — {format_datetime(s['time'])}"):
                if s["notes"]: st.write(f"_{s['notes']}_")
                if st.button("🗑️ Delete", key=f"del_symptom_{s['id']}"):
                    st.session_state.symptoms = [x for x in st.session_state.symptoms if x["id"] != s["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()
        st.markdown("---")
        st.subheader("🗑️ Auto-Clean Duplicates")
        if st.button("✨ Delete All Duplicate Symptoms (Keep First)", type="secondary"):
            seen = set()
            unique = []
            dup_count = 0
            for s in st.session_state.symptoms:
                key = (normalize(s["symptom"]), normalize(s["time"]), s["severity"])
                if key not in seen:
                    unique.append(s)
                    seen.add(key)
                else:
                    dup_count += 1
            st.session_state.symptoms = unique
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Deleted {dup_count} duplicates!") if dup_count else st.info("No duplicates.")
            st.rerun()

    # INSIGHTS TAB
    with tabs[3]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log Yoruba meals and symptoms to see culturally relevant insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

    # EXPORT TAB
    with tabs[4]:
        st.header("📤 Export")
        if st.session_state.meals:
            st.download_button("📥 Meals (CSV)", pd.DataFrame(st.session_state.meals).to_csv(index=False), "meals.csv")
        if st.session_state.symptoms:
            st.download_button("📥 Symptoms (CSV)", pd.DataFrame(st.session_state.symptoms).to_csv(index=False), "symptoms.csv")

else:
    # USER VIEW
    with tabs[1]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log Yoruba meals and symptoms to see culturally relevant insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

Overwriting welltrack_app.py


In [17]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"\n✅ WellTrack (Yoruba-aware) is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &


✅ WellTrack (Yoruba-aware) is live at:
👉 NgrokTunnel: "https://trisyllabical-semitruthfully-junko.ngrok-free.dev" -> "http://localhost:8501"
🔓 Developer code: WELLTRACK_DEV_2025


In [18]:
%%writefile welltrack_app.py
import streamlit as st
import json
import os
import pandas as pd
from datetime import datetime, timedelta

DATA_FILE = "/content/welltrack_data.json"
DEVELOPER_SECRET = "WELLTRACK_DEV_2025"

# ============================
# FOOD KNOWLEDGE BASE (Yoruba + Global)
# ============================
FOOD_KB = {
    "legume": ["bean", "ewa", "moimoo", "moi moi", "aganyin", "soya", "lentil", "chickpea"],
    "fried": ["akara", "fried yam", "puff-puff", "bun", "spring roll", "fried plantain"],
    "palm_oil": ["ayamase", "egusi", "efo riro", "buka stew", "ofada stew", "palm oil", "ogbono"],
    "swallow": ["amala", "eba", "fufu", "pounded yam", "lafun", "semo", "wheat", "tuwo", "ogi"],
    "whole": ["oatmeal", "salmon", "mackerel", "brown rice", "vegetables", "fruits", "nuts", "yam porridge"],
    "dairy": ["milk", "cheese", "yogurt", "ice cream", "butter", "cream"],
    "spicy": ["pepper", "chili", "spicy", "hot sauce", "buka"]
}

def normalize(val):
    return str(val).strip().lower() if val is not None else ""

def classify_meal(food_str):
    food = normalize(food_str)
    categories = set()
    for cat, keywords in FOOD_KB.items():
        if any(kw in food for kw in keywords):
            categories.add(cat)
    return list(categories)

def load_data():
    if os.path.exists(DATA_FILE):
        try:
            with open(DATA_FILE, "r") as f:
                data = json.load(f)
                meals = data.get("meals", [])
                symptoms = data.get("symptoms", [])
                for i, m in enumerate(meals):
                    if "id" not in m: m["id"] = -1_000_000 - i
                for i, s in enumerate(symptoms):
                    if "id" not in s: s["id"] = -2_000_000 - i
                return meals, symptoms
        except:
            pass
    return [], []

def save_data(meals, symptoms):
    with open(DATA_FILE, "w") as f:
        json.dump({"meals": meals, "symptoms": symptoms}, f)

def is_meal_duplicate(new_meal, meals):
    key = (normalize(new_meal["food"]), normalize(new_meal["time"]), normalize(new_meal["portion"]))
    return any((normalize(m["food"]), normalize(m["time"]), normalize(m["portion"])) == key for m in meals)

def is_symptom_duplicate(new_symptom, symptoms):
    key = (normalize(new_symptom["symptom"]), normalize(new_symptom["time"]), new_symptom["severity"])
    return any((normalize(s["symptom"]), normalize(s["time"]), s["severity"]) == key for s in symptoms)

def generate_insights(meals, symptoms):
    if not meals or not symptoms:
        return []

    # Build symptom profile
    symptom_text = " ".join(normalize(s["symptom"]) + " " + normalize(s.get("notes", "")) for s in symptoms)

    # Extract all food categories from recent meals
    all_categories = set()
    for m in meals[-10:]:  # last 10 meals
        all_categories.update(classify_meal(m["food"]))

    insights = []

    # 1. Legumes → Gas/Bloating
    if "legume" in all_categories and any(kw in symptom_text for kw in ["gas", "bloat", "cramp", "distension", "flatulence"]):
        insights.append({
            "food": "Legume-based foods (Beans, Ewa, Moimoo)",
            "symptoms": ["Gas", "Bloating", "Abdominal discomfort"],
            "confidence": "High",
            "action": "Soak beans for 12+ hours before cooking; add ginger, garlic, or cumin to reduce gas."
        })

    # 2. Fried/Palm Oil → Heartburn/Reflux
    if ("fried" in all_categories or "palm_oil" in all_categories) and any(kw in symptom_text for kw in ["heartburn", "reflux", "acid", "nausea", "burning"]):
        insights.append({
            "food": "Fried or palm oil-rich dishes (Akara, Egusi, Ayamase)",
            "symptoms": ["Heartburn", "Acid reflux", "Nausea"],
            "confidence": "High",
            "action": "Reduce oil by 50%; avoid eating within 3 hours of bedtime; use less pepper."
        })

    # 3. Swallow meals → Fullness/Drowsiness
    if "swallow" in all_categories and any(kw in symptom_text for kw in ["full", "drowsy", "heavy", "sluggish", "sleepy"]):
        insights.append({
            "food": "High-starch swallow meals (Amala, Eba, Pounded Yam)",
            "symptoms": ["Heaviness", "Drowsiness", "Slow digestion"],
            "confidence": "Medium",
            "action": "Reduce portion size; always pair with light, vegetable-based soup like Efo Riro."
        })

    # 4. Spicy foods → Stomach ache
    if "spicy" in all_categories and any(kw in symptom_text for kw in ["stomach", "ache", "pain", "discomfort", "upset"]):
        insights.append({
            "food": "Spicy or pepper-heavy dishes",
            "symptoms": ["Stomach ache", "Discomfort", "Upset stomach"],
            "confidence": "Medium",
            "action": "Moderate pepper and spice levels, especially if you have a sensitive stomach."
        })

    # 5. Whole foods → Positive outcomes
    if "whole" in all_categories and any(kw in symptom_text for kw in ["energy", "good digestion", "light", "well", "better"]):
        insights.append({
            "food": "Whole foods (Fish, Oats, Vegetables, Yam porridge)",
            "symptoms": ["Increased energy", "Better digestion", "Feeling light"],
            "confidence": "High",
            "action": "Continue incorporating these foods regularly — they support long-term gut health."
        })

    # 6. Dairy (if detected)
    if "dairy" in all_categories and any(kw in symptom_text for kw in ["stomach", "headache", "bloat", "discomfort"]):
        insights.append({
            "food": "Dairy products (Milk, Ice cream, Cheese)",
            "symptoms": ["Stomach discomfort", "Headache", "Bloating"],
            "confidence": "Medium",
            "action": "Consider a 2-week dairy elimination trial to confirm intolerance."
        })

    return insights

def format_datetime(dt_str):
    try:
        return datetime.fromisoformat(dt_str).strftime("%b %-d, %-I:%M %p")
    except:
        return dt_str

def in_date_range(item, start, end):
    try:
        dt = datetime.fromisoformat(item["time"])
        return start <= dt.date() <= end
    except:
        return True

# ============================
# INITIALIZE
# ============================
if "initialized" not in st.session_state:
    meals_init, symptoms_init = load_data()
    st.session_state.meals = meals_init
    st.session_state.symptoms = symptoms_init
    st.session_state.initialized = True

# ============================
# UI SETUP
# ============================
st.sidebar.title("🔐 Developer Access")
dev_code = st.sidebar.text_input("Enter code", type="password")
is_developer = (dev_code == DEVELOPER_SECRET)

tabs_list = ["Log", "Insights"]
if is_developer:
    tabs_list = ["Log", "Meals", "Symptoms", "Insights", "📤 Export"]

st.set_page_config(page_title="WellTrack", page_icon="❤️", layout="centered")
st.title("❤️ WellTrack")
st.caption("Track meals, symptoms & get intelligent health insights")

tabs = st.tabs(tabs_list)

# ============================
# LOG TAB
# ============================
with tabs[0]:
    st.header("📝 Daily Health Log")
    log_type = st.radio("Log", (" Meal 🍽️ ", " Symptom ⚠️ "), horizontal=True)

    if "Meal" in log_type:
        with st.form("meal_form"):
            food = st.text_input("Food", placeholder="e.g., Amala with Ewedu")
            portion = st.text_input("Portion", placeholder="e.g., 1 bowl")
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes", placeholder="How did you feel?")
            if st.form_submit_button("✅ Add Meal"):
                new_meal = {"id": int(datetime.now().timestamp()), "food": food, "portion": portion, "time": time, "notes": notes}
                if is_meal_duplicate(new_meal, st.session_state.meals):
                    st.warning("⚠️ Duplicate meal skipped.")
                else:
                    st.session_state.meals.insert(0, new_meal)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Meal logged!")
                st.rerun()
    else:
        with st.form("symptom_form"):
            symptom = st.text_input("Symptom", placeholder="e.g., Stomach bloating")
            severity = st.slider("Severity (1–10)", 1, 10, 5)
            time = st.text_input("Time (YYYY-MM-DDTHH:MM)", value=datetime.now().strftime("%Y-%m-%dT%H:%M"))
            notes = st.text_area("Notes", placeholder="e.g., After eating Ewa Aganyin")
            if st.form_submit_button("✅ Add Symptom"):
                new_symptom = {"id": int(datetime.now().timestamp()), "symptom": symptom, "severity": severity, "time": time, "notes": notes}
                if is_symptom_duplicate(new_symptom, st.session_state.symptoms):
                    st.warning("⚠️ Duplicate symptom skipped.")
                else:
                    st.session_state.symptoms.insert(0, new_symptom)
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.success("✅ Symptom logged!")
                st.rerun()

    st.markdown("### 📥 Upload Data")
    st.write("Upload your meal/symptom CSV for immediate insights.")
    uploaded = st.file_uploader("CSV file", type=["csv"])
    if uploaded:
        df = pd.read_csv(uploaded)
        added_meals = added_symptoms = 0
        if {"food", "time", "portion", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                meal = {"id": int(datetime.now().timestamp() + added_meals), "food": str(row["food"]), "portion": str(row.get("portion", "")), "time": str(row.get("time", "")), "notes": str(row.get("notes", ""))}
                if not is_meal_duplicate(meal, st.session_state.meals):
                    st.session_state.meals.insert(0, meal)
                    added_meals += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_meals} meals.")
            st.rerun()
        elif {"symptom", "severity", "time", "notes"}.issubset(df.columns):
            for _, row in df.iterrows():
                symptom = {"id": int(datetime.now().timestamp() + added_symptoms), "symptom": str(row["symptom"]), "severity": int(row.get("severity", 5)), "time": str(row.get("time", "")), "notes": str(row.get("notes", ""))}
                if not is_symptom_duplicate(symptom, st.session_state.symptoms):
                    st.session_state.symptoms.insert(0, symptom)
                    added_symptoms += 1
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success(f"✅ Added {added_symptoms} symptoms.")
            st.rerun()
        else:
            st.error("❌ Invalid format. Use templates below.")

    st.markdown("### 📄 Templates")
    col1, col2 = st.columns(2)
    col1.download_button("Meal Template", pd.DataFrame([{"food":"Amala with Ewedu","time":"2025-11-30T08:00","portion":"1 bowl","notes":"Felt heavy"}]).to_csv(index=False), "meal.csv")
    col2.download_button("Symptom Template", pd.DataFrame([{"symptom":"Stomach bloating","severity":6,"time":"2025-11-30T15:30","notes":"After Ewa Aganyin"}]).to_csv(index=False), "symptom.csv")

# ============================
# DEVELOPER TABS
# ============================
if is_developer:
    # MEALS
    with tabs[1]:
        st.header("Meal History")
        colf1, colf2 = st.columns(2)
        start = colf1.date_input("Start", value=datetime.today().date() - timedelta(days=30))
        end = colf2.date_input("End", value=datetime.today().date())
        filtered = [m for m in st.session_state.meals if in_date_range(m, start, end)]
        for meal in filtered:
            with st.expander(f"🍽️ {meal['food']} — {format_datetime(meal['time'])}"):
                st.write(f"**Portion:** {meal['portion']}")
                if meal["notes"]: st.write(f"_{meal['notes']}_")
                if st.button("🗑️ Delete", key=f"m_{meal['id']}"):
                    st.session_state.meals = [m for m in st.session_state.meals if m["id"] != meal["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()
        if st.button("✨ Delete All Duplicates", key="clean_meals"):
            seen = set()
            unique = []
            for m in st.session_state.meals:
                key = (normalize(m["food"]), normalize(m["time"]), normalize(m["portion"]))
                if key not in seen:
                    unique.append(m)
                    seen.add(key)
            st.session_state.meals = unique
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success("✅ Duplicates removed!")
            st.rerun()

    # SYMPTOMS
    with tabs[2]:
        st.header("Symptom Tracker")
        colf1, colf2 = st.columns(2)
        start = colf1.date_input("Start", value=datetime.today().date() - timedelta(days=30), key="s_start")
        end = colf2.date_input("End", value=datetime.today().date(), key="s_end")
        filtered = [s for s in st.session_state.symptoms if in_date_range(s, start, end)]
        for s in filtered:
            color = "🟢" if s["severity"] <= 3 else "🟡" if s["severity"] <= 7 else "🔴"
            with st.expander(f"{color} {s['symptom']} — {s['severity']}/10 — {format_datetime(s['time'])}"):
                if s["notes"]: st.write(f"_{s['notes']}_")
                if st.button("🗑️ Delete", key=f"s_{s['id']}"):
                    st.session_state.symptoms = [x for x in st.session_state.symptoms if x["id"] != s["id"]]
                    save_data(st.session_state.meals, st.session_state.symptoms)
                    st.rerun()
        if st.button("✨ Delete All Duplicates", key="clean_symptoms"):
            seen = set()
            unique = []
            for s in st.session_state.symptoms:
                key = (normalize(s["symptom"]), normalize(s["time"]), s["severity"])
                if key not in seen:
                    unique.append(s)
                    seen.add(key)
            st.session_state.symptoms = unique
            save_data(st.session_state.meals, st.session_state.symptoms)
            st.success("✅ Duplicates removed!")
            st.rerun()

    # INSIGHTS
    with tabs[3]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log meals and symptoms to see intelligent, personalized insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

    # EXPORT
    with tabs[4]:
        st.header("📤 Export")
        if st.session_state.meals:
            st.download_button("📥 Meals (CSV)", pd.DataFrame(st.session_state.meals).to_csv(index=False), "meals.csv")
        if st.session_state.symptoms:
            st.download_button("📥 Symptoms (CSV)", pd.DataFrame(st.session_state.symptoms).to_csv(index=False), "symptoms.csv")

else:
    # USER VIEW
    with tabs[1]:
        st.header("Health Insights")
        insights = generate_insights(st.session_state.meals, st.session_state.symptoms)
        if not insights:
            st.info("Log meals and symptoms to see intelligent, personalized insights!")
        else:
            for i in insights:
                icon = "✅" if i["confidence"] == "High" else "⚠️"
                st.subheader(f"{icon} {i['food']}")
                st.write(f"**Symptoms:** {', '.join(i['symptoms'])}")
                st.write(f"**Confidence:** {i['confidence']}")
                st.info(f"**Recommendation:** {i['action']}")
        st.caption("Note: This is for precautionary information only. Consult a health professional for critical issues.")

Overwriting welltrack_app.py


In [19]:
!pip install streamlit pyngrok pandas -q
!pkill ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"\n✅ WellTrack with adaptive insights is live at:\n👉 {public_url}")
print(f"🔓 Developer code: WELLTRACK_DEV_2025")
!streamlit run welltrack_app.py --server.port=8501 --server.address=0.0.0.0 >/dev/null 2>&1 &


✅ WellTrack with adaptive insights is live at:
👉 NgrokTunnel: "https://trisyllabical-semitruthfully-junko.ngrok-free.dev" -> "http://localhost:8501"
🔓 Developer code: WELLTRACK_DEV_2025
